In [1]:
import numpy as np

In [156]:
kernel = np.flip(2 ** np.arange(9)).reshape(3, 3)
kernel

array([[256, 128,  64],
       [ 32,  16,   8],
       [  4,   2,   1]])

In [159]:
np.array(key)[kernel]

array([[False, False, False],
       [ True, False,  True],
       [False,  True, False]])

In [198]:
def parse_int(s):
    return [c == '#' for c in s]

def parse_image(s):
    key, img = s.split('\n\n')
    key = np.array(parse_int(key.strip()))
    img = np.array([parse_int(l) for l in img.strip().splitlines()], dtype=bool)
    return key, img

def pad_image(img, padding=2, boundary=False):
    return np.pad(img, padding, mode='constant', constant_values=boundary)

def enhance_pixel(pix, key):
    return key[int(''.join('01'[int(i)] for i in pix.ravel()), 2)]

def pixel_windows(img):
    return np.lib.stride_tricks.sliding_window_view(img, (3, 3)).reshape((-1, 3, 3))

def enhance(img, key, boundary=False):
    padded = pad_image(img, boundary=boundary)
    new_boundary = bool(key[(0, -1)[boundary]])
    windows = (np.lib.stride_tricks.sliding_window_view(padded, (3, 3)) * kernel)
    new_image = key[windows.sum(axis=(2, 3))]
    return new_image, new_boundary

def enhance_n(img, key, n):
    boundary = False
    for _ in range(n):
        img, boundary = enhance(img, key, boundary)
    return img

def show_image(img):
    print('\n'.join(''.join('.#'[int(i)] for i in row) for row in img))

In [199]:
key, img = parse_image("""..#.#..#####.#.#.#.###.##.....###.##.#..###.####..#####..#....#..#..##..###..######.###...####..#..#####..##..#.#####...##.#.#..#.##..#.#......#.###.######.###.####...#.##.##..#..#..#####.....#.#....###..#.##......#.....#..#..#..##..#...##.######.####.####.#.#...#.......#..#.#.#...####.##.#......#..#...##.#.##..#...##.#.##..###.#......#.#.......#.#.#.####.###.##...#.....####.#..#..#.##.#....##..#.####....##...##..#...#......#.#.......#.......##..####..#...#.#.#...##..#.#..###..#####........#..####......#..#

#..#.
#....
##..#
..#..
..###""")

In [200]:
img = enhance_n(img, key, 1)
show_image(img)

.##.##.
#..#.#.
##.#..#
####..#
.#..##.
..##..#
...#.#.


In [201]:
img = enhance_n(img, key, 1)
show_image(img)

.......#.
.#..#.#..
#.#...###
#...##.#.
#.....#.#
.#.#####.
..#.#####
...##.##.
....###..


In [202]:
img = enhance_n(img, key, 50)
print(img.sum())

3565


In [203]:
%%time
with open('../data/day20.txt') as infile:
    key, img = parse_image(infile.read())
    print('[p1] Number of lit pixels after 2:', enhance_n(img, key, 2).sum())    
    print('[p2] Number of lit pixels after 50:', enhance_n(img, key, 50).sum())

[p1] Number of lit pixels after 2: 5057
[p2] Number of lit pixels after 50: 18502
CPU times: user 122 ms, sys: 4.04 ms, total: 126 ms
Wall time: 123 ms
